In [ ]:
%%javascript
// to avoid scroll in windows
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from IPython.display import Markdown as md
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
def day(n):
    return datetime(2020, 2, 3) + timedelta(n)

In [ ]:
rundf = pd.read_csv('results/'+
        'SIsaR_0.9.5.2 experiments 2 seeds-table_1000_basic_control_schoolOnSept.csv',
                    sep=',',skiprows=6)
case="# 1000 epidemics with non-pharmaceutical containment measures, "+\
          "schools open in Sep, changing world, using checkpoints"
md(case)

In [ ]:
rundf["totalInfected"]=rundf["cumulativeRedCount"]+rundf["cumulativeVioletCount"]
rundf["totalInfected&Deceased"]=rundf["totalInfected"]+rundf["cumulativeDeadCount"]

rundf["cp1totalInfected"]=rundf["cp1cRed"]+rundf["cp1cViolet"]
rundf["cp1totalInfected&Deceased"]=rundf["cp1totalInfected"]+rundf["cp1cDead"]

rundf["cp2totalInfected"]=rundf["cp1cRed"]+rundf["cp2cViolet"]
rundf["cp2totalInfected&Deceased"]=rundf["cp2totalInfected"]+rundf["cp1cDead"]

rundf["cp3totalInfected"]=rundf["cp1cRed"]+rundf["cp3cViolet"]
rundf["cp3totalInfected&Deceased"]=rundf["cp3totalInfected"]+rundf["cp1cDead"]

rundf["cp4totalInfected"]=rundf["cp1cRed"]+rundf["cp4cViolet"]
rundf["cp4totalInfected&Deceased"]=rundf["cp4totalInfected"]+rundf["cp1cDead"]

rundf["cp5totalInfected"]=rundf["cp1cRed"]+rundf["cp5cViolet"]
rundf["cp5totalInfected&Deceased"]=rundf["cp5totalInfected"]+rundf["cp1cDead"]

rundf["cp6totalInfected"]=rundf["cp1cRed"]+rundf["cp6cViolet"]
rundf["cp6totalInfected&Deceased"]=rundf["cp6totalInfected"]+rundf["cp1cDead"]

In [ ]:
rundf.columns

In [ ]:
rundf.replace(-1, np.nan, inplace=True)

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
rundf[['cp1cRedNH','cumulativeRedCount','totalInfected&Deceased','ticks']].describe()

In [ ]:
tmp=rundf[['cp1cRedNH','cumulativeRedCount','totalInfected&Deceased','ticks']].describe()

In [ ]:
print (tmp.to_latex())

The output is included within:  
\begin{table}[htbp] or [H]

... output above ...

\label{a label}  
\caption{a caption}  
\end{table}


## 1 simulation time

In [ ]:
rundf.hist(column="ticks",bins=1000,figsize=(15,8))

## 2 symptomatic agent cumulative count

In [ ]:
rundf.hist(column="cumulativeRedCount",bins=1000,figsize=(15,8))

## 3 symptomatic agent cumulative count vs. time 

In [ ]:
rundf.plot(x="ticks",y="cumulativeRedCount",s=3,kind='scatter',figsize=(15,8), 
           title="symptomatic agent cumulative count vs. time",
           ylim=(0,3600),xlim=(0,700))

## 4 symptomatic + asymptomatic agent cumulative count

In [ ]:
rundf.hist(column="totalInfected",bins=1000,figsize=(15,8))

## 5 total number of deceased

In [ ]:
rundf.hist(column="cumulativeDeadCount",bins=1000,figsize=(15,8))

## 6 symptomatic + asymptomatic + deceased agent cumulative count

In [ ]:
rundf.hist(column="totalInfected&Deceased",bins=1000,figsize=(15,8))

## 7 symptomatic + asymptomatic + deceased agent cumulative count vs. time

In [ ]:
rundf.plot(x="ticks",y="totalInfected&Deceased",s=3,kind='scatter',figsize=(15,8), 
           title="simptomatic+asymptomatic+deceased vs. time",ylim=(0,3600),xlim=(0,700))

In [ ]:
md("# ------------------------------------------------------------\n"+\
    "# timeFunction displaying heat-maps and interval data\n"+\
    "### with the possibility of choosing the duration column (after a merge), if any\n"+\
    "# checkpoint function displaying the data\n"+\
    "### of the epdicemics subsisting at a given checkpoint\n"+\
    "# intervalCheckpoint function displaying the data\n"+\
    "### of the epdicemics subsisting in an interval and at a given checkpoint\n"+\
    "# ------------------------------------------------------------")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import LogNorm
import seaborn as sns; sns.set()

def timeFunction(day0,day1,durationCol,title):
    
    rundfHere=rundf.copy()
    rundfHere=rundfHere[rundfHere[durationCol]>=day0]
    rundfHere=rundfHere[rundfHere[durationCol]<=day1]
    
    display(md ("## Epidemics with duration between {} days ({}) and {} days ({})"\
                   .format(day0,day(day0).strftime("%Y-%m-%d"),\
                           day1,day(day1).strftime("%Y-%m-%d"))) )
    
    
    rundfHere.rename(columns={'cumulativeRedCount': 'symptomatic',
                              'ticks':"duration"},inplace=True)
    
    tmp=rundfHere[['symptomatic','totalInfected&Deceased','duration']].describe();
    display(tmp)
    display(print(tmp.to_latex()))
    
    if rundfHere.shape[0] == 0: return
    
    for nBins in [10,20,30,60]:
        display(md ("## Epidemics with duration between {} days ({}) and {} days ({}), with {} bins"\
                   .format(day0,day(day0).strftime("%Y-%m-%d"),\
                           day1,day(day1).strftime("%Y-%m-%d"),nBins)) )
            
        rundfHere['totalInfected&Deceased'].plot.hist(bins=nBins, color="r",
                                            title=title+"\ntotalInfected&Deceased");
        plt.show()
        rundfHere['duration'].plot.hist(bins=nBins, color="g",
                                            title=title+"\ntotalInfected&Deceased");
        plt.show()

        # preparing data for the heat-map
        hist = plt.hist2d(x=rundfHere['duration'], y=rundfHere['totalInfected&Deceased'], 
                          bins=nBins, range=[[0, 700], [0, 3600]])
        plt.close() #to avoid the output of hist2d
        
        zMax=np.amax(hist[0]) # max level z axis
        display(md ("With {} bins, the max value on z axis is {}, first row is {}"\
                    .format(nBins,zMax,np.transpose(hist[0])[0])))
        
        if nBins == 60: plt.figure(figsize=(20,10))
        else:           plt.figure(figsize=(8.5,4.25))
        
        lognorm = LogNorm(vmin=1.0, vmax=np.amax(hist[0]))
        ax = sns.heatmap(
            np.flip(np.transpose(hist[0]),0), 
            #robust=True, 
            norm=lognorm,
            cmap='YlGnBu',
            annot=True, fmt='.0f', 
            xticklabels=np.round(hist[1][1:],0).astype(int),
            yticklabels=np.flip(hist[2][1:],0).astype(int),
            annot_kws={"size":8}
            )

        plt.xlabel("Days")
        plt.ylabel("Total symptomatic + asymptomatic + deceased")

        plt.title(title)

        plt.show()


In [ ]:
#checkpoint names
start = 0
end=1000
cp1=[1, 30,"cp1cRed","cp1totalInfected&Deceased"]
cp2=[2,119,"cp2cRed","cp2totalInfected&Deceased"]
cp3=[3,230,"cp3cRed","cp3totalInfected&Deceased"]
cp4=[4,316,"cp4cRed","cp4totalInfected&Deceased"]
cp5=[5,364,"cp5cRed","cp5totalInfected&Deceased"]
cp6=[6.453,"cp6cRed","cp6totalInfected&Deceased"]

In [ ]:
def checkPoint(cpN,durationCol,title):
    
    # epidemics subsisting a cpN[1] day
    display(md("## "+title))
    display(md("## Checkpoint {}, data of the epidemics subsisting at day {}"\
               .format(cpN[0],cpN[1])))
    
    rundfHere2=rundf.copy()
    #display(rundfHere.columns)
    rundfHere2=rundfHere2[rundfHere2[durationCol]>=cpN[1]]

    rundfHere2.rename(columns={cpN[2]: 'symptomatic-cp'+str(cpN[0]),
                               cpN[3]: 'totalInfected&Deceased-cp'+str(cpN[0]),
                               "ticks": "duration"},inplace=True)
    
    tmp=rundfHere2[['symptomatic-cp'+str(cpN[0]),'totalInfected&Deceased-cp'+str(cpN[0]),\
                    'duration']].describe();
    display(tmp)
    display(print(tmp.to_latex()))

In [ ]:
def checkPointInterval(day0,day1,cpN,durationCol,title):

    display(md("## "+title))

    display(md ("## Epidemics with duration between {} days ({}) and {} days ({})\n## +"\
                   .format(day0,day(day0).strftime("%Y-%m-%d"),\
                           day1,day(day1).strftime("%Y-%m-%d"))) )
    display(md ("## Checkpoint {}, data of the epidemics subsisting at day {}"\
                   .format(cpN[0],cpN[1])) )

    rundfHere1=rundf.copy()
    rundfHere1=rundfHere1[rundfHere1[durationCol]>=day0]
    rundfHere1=rundfHere1[rundfHere1[durationCol]<=day1]

    rundfHere1.rename(columns={'cumulativeRedCount': 'symptomatic'},inplace=True)
    
    #display(rundfHere1[['symptomatic','totalInfected&Deceased']].describe())
    
    rundfHere2=rundf.copy()
    rundfHere2=rundfHere2.drop(['totalInfected&Deceased'], axis=1)
    rundfHere2=rundfHere2[rundfHere2[durationCol]>=cpN[1]]
    rundfHere2.rename(columns={cpN[2]: 'symptomatic',
                               cpN[3]: 'totalInfected&Deceased'},inplace=True)
    #display(rundfHere2[['symptomatic','totalInfected&Deceased']].describe())
    
    rundfHere12=pd.DataFrame()
    rundfHere12[['symptomatic','totalInfected&Deceased']]=\
          rundfHere1[['symptomatic','totalInfected&Deceased']]\
          .append(rundfHere2[['symptomatic','totalInfected&Deceased']])
    display(rundfHere12[['symptomatic','totalInfected&Deceased']].describe())


In [ ]:
checkPoint(cp1,"ticks",'1000 epidemics with non-pharmaceutical containment measures, '+
          'schools open in Sep')

In [ ]:
timeFunction(start,cp1[1]-1,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

In [ ]:
timeFunction(cp1[1],end,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

#### --------------------------------------------------

In [ ]:
checkPoint(cp2,"ticks",'1000 epidemics with non-pharmaceutical containment measures, '+
          'schools open in Sep')

In [ ]:
timeFunction(start,cp2[1]-1,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

In [ ]:
timeFunction(cp1[1],cp2[1]-1,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

In [ ]:
timeFunction(cp2[1],end,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

In [ ]:
checkPointInterval(start,cp2[1]-1,cp2,"ticks",
                   '1000 epidemics with non-pharmaceutical containment measures, '+
                   'schools open in Sep')

#### --------------------------------------------------

In [ ]:
checkPoint(cp3,"ticks",'1000 epidemics with non-pharmaceutical containment measures, '+
          'schools open in Sep')

In [ ]:
timeFunction(start,cp3[1]-1,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

In [ ]:
timeFunction(cp2[1],cp3[1]-1,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

In [ ]:
timeFunction(cp3[1],end,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

In [ ]:
checkPointInterval(cp2[1],cp3[1]-1,cp3,"ticks",
                   '1000 epidemics with non-pharmaceutical containment measures, '+
                   'schools open in Sep')

#### --------------------------------------------------

In [ ]:
checkPoint(cp4,"ticks",'1000 epidemics with non-pharmaceutical containment measures, '+
          'schools open in Sep')

In [ ]:
timeFunction(start,cp4[1]-1,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

In [ ]:
timeFunction(cp3[1],cp4[1]-1,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

In [ ]:
timeFunction(cp4[1],end,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

In [ ]:
checkPointInterval(cp3[1],cp4[1]-1,cp4,"ticks",
                   '1000 epidemics with non-pharmaceutical containment measures, '+
                   'schools open in Sep')

#### --------------------------------------------------

In [ ]:
checkPoint(cp5,"ticks",'1000 epidemics with non-pharmaceutical containment measures, '+
          'schools open in Sep')

In [ ]:
timeFunction(start,cp5[1]-1,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

In [ ]:
timeFunction(cp4[1],cp5[1]-1,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

In [ ]:
timeFunction(cp5[1],end,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

In [ ]:
checkPointInterval(cp4[1],cp5[1]-1,cp5,"ticks",
                   '1000 epidemics with non-pharmaceutical containment measures, '+
                   'schools open in Sep')

#### --------------------------------------------------

In [ ]:
checkPoint(cp6,"ticks",'1000 epidemics with non-pharmaceutical containment measures, '+
          'schools open in Sep')

In [ ]:
timeFunction(start,cp6[1]-1,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

In [ ]:
timeFunction(cp5[1],cp6[1]-1,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

In [ ]:
timeFunction(cp6[1],end,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')

#### --------------------------------------------------

In [ ]:
timeFunction(start,end,"ticks",'1000 epidemics with non-pharmaceutical containment measures,\n'+
          'schools open in Sep')